![miad4.png](Archivos/miad4.png)

In [ ]:
import os
# Por precaución, cambiamos el directorio activo de Python a aquel que contenga este notebook
if "PAD-book" in os.listdir():
    os.chdir(r"PAD-book/Laboratorio-Computacional-de-Analytics/S4 - Explorar, modificar y visualizar bases de datos/S4.TU2/")

# Modificar bases de datos  

Al finalizar la exploración de una base de datos procedemos a modificarla. En este tutorial expandimos la explicación de los métodos de la librería `pandas` vistos anteriormente y presentamos métodos adicionales para integrarlos en procesos de modificación de bases de datos. 

## Requisitos

Para desarrollar este tutorial necesitarás:

* Importar archivos de texto.
* Implementar filtrado condicional y filtrado numérico.
* Utilizar los métodos básicos de los `Series` y los `DataFrame`.

## Objetivos

Al final de este tutorial podrás:

**1.** Procesar archivos en formatos especiales. <br>
**2.** Modificar la estructura y el contenido de un `DataFrame`.

## 1. Procesamiento de archivos en formatos especiales 

La librería `pandas` nos permite importar archivos en múltiples formatos para procesarlos como un `DataFrame`. Entre estos formatos, se encuentran los archivos de tipo texto, de tipo binario y de tipo SQL. En las referencias encuentras la lista completa de métodos de lectura y escritura según el tipo de archivo. A manera de ejemplo expondremos el método `read_fwf` el cuál permite procesar archivos de ancho fijo. 

```python
pd.read_fwf(filepath_or_buffer, colspecs, **kwds)
```

* **filepath_or_buffer:** ruta del archivo de texto a procesar.<br><br>

* **colspecs:** lista de tuplas que identifica la posición donde empieza y termina cada columna. Especificando `colspecs = 'infer'` el método infiere las columnas del archivo.<br><br>

* ** **kwds:** indica que podemos especificar los parámetros disponibles en el método `read_csv`. 

### Ejemplo 1 

Retomemos el archivo `"mfh.txt"` (*Major foreign holders*) el cual contiene la distribución del monto de títulos del tesoro americano de los Estados Unidos que se encuentra en cuentas afuera de su país. Dado que este es un archivo de ancho fijo, utilizaremos parte del código que aprendimos anteriormente para obtener las posiciones de inicio y de fin de las columnas que conforman el archivo.

In [1]:
with open("Archivos/mfh.txt", "r") as mfh: 
    # Leemos el archivo y lo almacenamos en una sola cadena de caracteres
    mfh = mfh.read()
    # Seleccionamos la cadena de caracteres que se encuentra entre la palabra 'Period' y la palabra 'Grand'
    mfh = mfh.split('PERIOD')[1].split('Grand')[0]
    # Declaramos una lista con las líneas de nuestra cadena de caracteres
    lista_mfh = mfh.splitlines()
    # Nos quedamos únicamente con las líneas no vacías
    lista_mfh = [x for x in lista_mfh if x]

Declaramos las siguientes variables: 

In [2]:
# Lista vacía para almacenar las posiciones de inicio de cada columna de números.
lista_inicio_cols = []
# Lista vacía para almacenar la última posición de cada columna de números. 
lista_fin_cols = []
# Cadena de caracteres que almacena la fila de guíones contenida en el archivo.
fila_guiones = lista_mfh[2]
# Variable de tipo entero para marcar el inicio de la busqueda a partir de cada columna. 
inicio_columna = 0
# Variable de tipo lógica para indicar si ya llegamos al final de la lista.
criterio_parada = True

Iteramos mientras el booleano se mantenga igual a `True`. 

In [3]:
while(criterio_parada):
    # Declaramos el inicio de una nueva columna encontrando la palabra guión.
    pos_inicio = inicio_columna + fila_guiones[inicio_columna:].find('-')
    # Declaramos el ancho de la columna encontrando el siguiente espacio vacío.  
    ancho_columna = fila_guiones[pos_inicio:].find(' ')
    
    # Si encuentra dicho espacio
    if(ancho_columna != -1):
        # Almacenamos la posición de inicio de la columna.
        lista_inicio_cols.append(pos_inicio)
        # Declaramos la posición final de la columna.
        pos_final = pos_inicio + ancho_columna 
        # Almacenamos la posición final de la columna.
        lista_fin_cols.append(pos_final) 
        # Actualizamos el inicio de la columna para la siguiente iteración.
        inicio_columna = pos_final
    #Si no se pudo encontrar mas espacios no se se sigue iterando ya que no hay mas columnas.    
    else:
        criterio_parada = False

El código anterior nos permite obtener las posiciones en las que empiezan y finalizan las columnas de números. Entonces agreguemos la tupla que contiene la posición donde empieza la columna de países (posición 0) y la posición donde empieza la primera columna de números.

In [4]:
ancho_cols = [(0,lista_inicio_cols[0])]+list(zip(lista_inicio_cols, lista_fin_cols))
ancho_cols

[(0, 32),
 (32, 38),
 (40, 46),
 (48, 54),
 (56, 62),
 (64, 70),
 (72, 78),
 (80, 86),
 (88, 94),
 (96, 102),
 (104, 110),
 (112, 118),
 (120, 126)]

Importemos las librerías necesarias.

In [5]:
import numpy as np 
import pandas as pd

Luego, utilizamos el método `read_fwf` para leer el archivo, omitiendo las filas correspondientes al encabezado y al pie de tabla. Además, para efectos de este tutorial debemos omitir los registros correspondientes al gran recuento total (`'Grand Total'`) y los demás países (`'All other'`).

In [6]:
mfh = pd.read_fwf("Archivos/mfh.txt", skiprows = range(8), skipfooter = 21,
                  colspecs = ancho_cols, index_col = [0], header = [0,1])

# Eliminamos las dos primeras entradas ya que no contienen información relevante. 
mfh = mfh[1:-2]
mfh

,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
Country,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Japan,1276.2,1278.4,1293.0,1261.5,1260.4,1266.5,1272.6,1268.6,1211.8,1155.2,1160.6,1168.5
"China, Mainland",1061.7,1068.0,1073.4,1074.4,1083.7,1072.8,1081.6,1092.3,1078.6,1069.9,1089.1,1101.5
United Kingdom,428.9,419.9,424.7,445.6,445.8,429.2,469.7,477.1,450.3,392.1,400.5,412.7
Ireland,315.8,335.3,330.8,330.3,324.2,301.3,271.6,282.8,271.7,281.9,289.7,285.4
Brazil,265.1,265.0,265.7,264.1,264.4,259.5,264.4,285.9,283.3,281.8,293.3,298.5
Luxembourg,262.5,268.8,264.7,267.6,262.7,265.5,246.1,260.8,255.2,254.6,261.9,263.0
Switzerland,255.3,253.1,250.7,247.4,243.1,241.3,244.6,243.7,238.1,237.5,233.4,233.2
Hong Kong,245.5,250.9,267.1,266.4,269.0,259.4,256.0,268.4,253.0,249.7,249.7,243.6
Cayman Islands,231.6,228.9,222.3,224.0,216.7,213.0,209.4,228.2,225.1,238.2,234.2,236.8


Al observar el `DataFrame` de la celda anterior nos damos cuenta que sería deseable rotular correctamente los niveles de las columnas y el índice, para esto podemos usar el método `rename_axis`: 

In [7]:
mfh = mfh.rename_axis("Pais")
mfh = mfh.rename_axis(["Mes","Año"], axis = 1)
mfh

Mes,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,,,,,,,,,,,,
Japan,1276.2,1278.4,1293.0,1261.5,1260.4,1266.5,1272.6,1268.6,1211.8,1155.2,1160.6,1168.5
"China, Mainland",1061.7,1068.0,1073.4,1074.4,1083.7,1072.8,1081.6,1092.3,1078.6,1069.9,1089.1,1101.5
United Kingdom,428.9,419.9,424.7,445.6,445.8,429.2,469.7,477.1,450.3,392.1,400.5,412.7
Ireland,315.8,335.3,330.8,330.3,324.2,301.3,271.6,282.8,271.7,281.9,289.7,285.4
Brazil,265.1,265.0,265.7,264.1,264.4,259.5,264.4,285.9,283.3,281.8,293.3,298.5
Luxembourg,262.5,268.8,264.7,267.6,262.7,265.5,246.1,260.8,255.2,254.6,261.9,263.0
Switzerland,255.3,253.1,250.7,247.4,243.1,241.3,244.6,243.7,238.1,237.5,233.4,233.2
Hong Kong,245.5,250.9,267.1,266.4,269.0,259.4,256.0,268.4,253.0,249.7,249.7,243.6


## 2. Métodos para la modificación de bases de datos  

### 2.1. Método `set_index` 

Permite utilizar una o varias columnas para sustituir o modificar el índice de un `DataFrame`. Además de columnas podemos utilizar arreglos de `pandas` (`Series` e `Index`) o de arreglos de `numpy`. 

```python
pandas.DataFrame.set_index(keys, drop, append, inplace, verify_integrity)
```

* **keys:** indica la(s) llave(s) a usar como nuevo índice.<br><br>

* **drop:** elimina del `DataFrame` las columnas que se emplearon como llaves en el método. Por defecto es `True`.<br><br>

* **append:** preseva el índice original de la base de datos y le agrega la llave ingresada por parámetro. Por defecto es `False`.<br><br>

* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>
    
* **verify_integrity:** solo aplica si ignore_index = False. Por defecto, `verify_integrity = False`.<br><br>
    * `verify_integrity = True`: arroja ValueError si en el DataFrame resultante hay indices duplicados.<br>
    * `verify_integrity = False`: permite tener índices repetidos en el DataFrame resultante.

**Nota:** si utilizamos un arreglo externo al `DataFrame`, debemos asegurarnos que tenga el mismo tamaño que el que tiene actualmente el `DataFrame`.

#### Ejemplo 2

Requerimos analizar como se distribuye la tenencia de los títulos del tesoros entre dos grupos de países: los países miembros de la OECD y los países llamados paraísos fiscales. Ambos grupos no son mutuamente excluyentes por lo que pueden haber coincidencias entre ambos. A continuación, se encuentra declarada una lista para cada grupo de países.

In [8]:
lista_paraisos_fiscales = ["Belgium", "Ireland", "Luxembourg", "Malta", "Netherlands", "Cayman Island",
                           "Singapore", "Puerto Rico", "Hong Kong", "Switzerland"]

lista_oecd = ["Austria", "Australia", "Belgium", "Canada", "Chile", "Colombia", "Czech Republic", "Denmark", 
              "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Israel", 
              "Italy", "Japan", "Korea", "Latvia", "Lithuania", "Luxembourg", "Mexico",  "Netherlands", 
              "New Zealand", "Norway", "Poland", "Portugal", "Slovak Republic", "Slovenia", "Spain", "Sweden", 
              "Switzerland", "Turkey", "United Kingdom", "United States"]

Creamos las columnas correspondientes para posteriormente añadirlas al índice.

In [9]:
# Creamos las columnas "Clasificación Fiscal" y "OECD" 
mfh.loc[:,'Clasificación_Fiscal'] = "Tributación Regular"
mfh.loc[:,'OECD'] = "No pertenece"

# Definimos cuales países son considerado paraisos fiscales y cuáles son miembros de la OECD
mfh.loc[mfh.index.isin(lista_paraisos_fiscales), 'Clasificación_Fiscal'] = "Paraiso Fiscal"
mfh.loc[mfh.index.isin(lista_oecd), 'OECD'] = "Pertenece"

# Utilizamos set_index para añadir estas columnas al índice
mfh.set_index(keys =['Clasificación_Fiscal','OECD'], append = True, inplace = True)
mfh

,,Mes,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
,,Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,1276.2,1278.4,1293.0,1261.5,1260.4,1266.5,1272.6,1268.6,1211.8,1155.2,1160.6,1168.5
"China, Mainland",Tributación Regular,No pertenece,1061.7,1068.0,1073.4,1074.4,1083.7,1072.8,1081.6,1092.3,1078.6,1069.9,1089.1,1101.5
United Kingdom,Tributación Regular,Pertenece,428.9,419.9,424.7,445.6,445.8,429.2,469.7,477.1,450.3,392.1,400.5,412.7
Ireland,Paraiso Fiscal,Pertenece,315.8,335.3,330.8,330.3,324.2,301.3,271.6,282.8,271.7,281.9,289.7,285.4
Brazil,Tributación Regular,No pertenece,265.1,265.0,265.7,264.1,264.4,259.5,264.4,285.9,283.3,281.8,293.3,298.5
Luxembourg,Paraiso Fiscal,Pertenece,262.5,268.8,264.7,267.6,262.7,265.5,246.1,260.8,255.2,254.6,261.9,263.0
Switzerland,Paraiso Fiscal,Pertenece,255.3,253.1,250.7,247.4,243.1,241.3,244.6,243.7,238.1,237.5,233.4,233.2
Hong Kong,Paraiso Fiscal,No pertenece,245.5,250.9,267.1,266.4,269.0,259.4,256.0,268.4,253.0,249.7,249.7,243.6


### 2.2. Método `reset_index`

Resetea el índice de un `DataFrame`. Si se trata de un multi-índice permite eliminar uno o más niveles, dejando la opción de preservarlos como columnas.

```python
pandas.DataFrame.reset_index(level, drop)
```

* **level:** etiqueta o índice del multi-índice a eliminar. <br><br>

* **drop:** por defecto es `False` <br><br>
`Drop = True`: se eliminan los niveles. <br><br>
`Drop = False`: los niveles pasan a ser una columna. 

### 2.3. Método `drop`

Permite eliminar filas o columnas especificando su nombre o su índice. 

```python
pandas.DataFrame.drop(labels, axis, index, columns, level, inplace, errors)
```

* **labels:** indica el nombre o la posición de las filas o columnas a eliminar.<br><br>

* **axis:** por defecto es 0.<br><br>
    * `axis = 1`: indica que vamos a eliminar columnas. <br>
    * `axis = 0`: indica que vamos a eliminar filas.<br><br>

* **index:** se utiliza únicamente para eliminar filas, especificando el nombre o el índice de las filas a eliminar. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `labels` y `axis`.<br><br>

* **columns:** se utiliza únicamente para eliminar columnas, especificando el nombre o el índice de las columnas a eliminar. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `labels` y `axis`.<br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le aplicará el método.<br><br>

* **inplace:** por defecto es `False`.<br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones.

#### Ejemplo 3

Requerimos eliminar las observaciones del `DataFrame` `mfh` correspondientes al año 2020, y almacenar el `DataFrame` resultante en un `DataFrame` llamado `mfh_2021`.

Procedemos entonces utilizando el método `drop`.

In [10]:
mfh_2021 = mfh.drop(labels = '2020', axis = 1, level = 1 )  
mfh_2021

,,Mes,Dec,Nov,Oct
,,Año,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,
Japan,Tributación Regular,Pertenece,1155.2,1160.6,1168.5
"China, Mainland",Tributación Regular,No pertenece,1069.9,1089.1,1101.5
United Kingdom,Tributación Regular,Pertenece,392.1,400.5,412.7
Ireland,Paraiso Fiscal,Pertenece,281.9,289.7,285.4
Brazil,Tributación Regular,No pertenece,281.8,293.3,298.5
Luxembourg,Paraiso Fiscal,Pertenece,254.6,261.9,263.0
Switzerland,Paraiso Fiscal,Pertenece,237.5,233.4,233.2
Hong Kong,Paraiso Fiscal,No pertenece,249.7,249.7,243.6


### 2.4. Método `Groupby`

El método groupby permite realizar las siguientes tres operaciones a la vez: 

1. Separar en grupos el `DataFrame` de acuerdo a un criterio.
2. Aplicar una función a cada grupo. 
3. Combinar los resultados en un nuevo `DataFrame`.

La sintaxis para usar groupby es la siguiente:

```python
pandas.DataFrame.groupby(by, axis, level)
```

* **by:** indica el criterio para realizar el paso (1). Puede usarse un diccionario, un `Series`, entre otros.<br><br>

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`: indica que vamos a agrupar columnas.<br><br>
    * `axis = 0`: indica que vamos a agrupar filas.<br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le aplicará el método.

#### Ejemplo 4

Requerimos un `DataFrame` llamado `mfh_resumen` que sume el total de títulos del tesoro para los niveles del índice múltiple. 

Transformamos los valores de nuestro `DataFrame` a `float` y después aplicamos el método `groupby`.

In [11]:
mfh = mfh.astype(float)
mfh_resumen = mfh.groupby(level = ['Clasificación_Fiscal','OECD']).sum()
mfh_resumen

Mes                                   Sep     Aug     Jul     Jun     May  \
Año                                  2020    2020    2020    2020    2020   
Clasificación_Fiscal OECD                                                   
Paraiso Fiscal       No pertenece   404.5   411.2   425.7   416.9   411.1   
                     Pertenece     1120.9  1140.7  1129.3  1133.4  1110.0   
Tributación Regular  No pertenece  2430.2  2417.7  2395.9  2372.5  2351.6   
                     Pertenece     2564.4  2561.6  2579.7  2572.2  2553.6   

Mes                                   Apr     Mar     Feb     Jan     Dec  \
Año                                  2020    2020    2020    2020    2019   
Clasificación_Fiscal OECD                                                   
Paraiso Fiscal       No pertenece   405.5   407.5   433.8   413.7   397.6   
                     Pertenece     1084.5  1037.5  1076.2  1039.2  1046.5   
Tributación Regular  No pertenece  2318.8  2363.9  2481.1  2446.6  2435.9   
                     Pertenece     2550.4  2607.9  2660.5  2564.2  2422.0   

Mes                                   Nov     Oct  
Año                                  2019    2019  
Clasificación_Fiscal OECD                          
Paraiso Fiscal       No pertenece   400.3   382.4  
                     Pertenece     1049.6  1048.9  
Tributación Regular  No pertenece  2453.7  2474.9  
                     Pertenece     2449.8  2488.3

### 2.5. Método `apply`

El método `apply` lleva a cabo una función bien sea sobre las columnas o sobre las filas de un `DataFrame`. 

```python
pandas.DataFrame.apply(func, axis)
```

* **func:** función a aplicar. <br><br>

* **axis:**  por defecto es 0. <br><br>

    * `axis = 1`: indica que vamos a aplicar la función sobre las columnas.<br><br>
    * `axis = 0`: indica que vamos a aplicar la función sobre las filas.<br><br>

A continuación explicaremos tres ventajas que presenta el método `apply`: 

#### Trabajar con funciones declaradas previamente 

El parámetro `func` puede tomar el valor de una función declarada previamente.

##### Ejemplo 5

Requerimos calcular el porcentaje de títulos distribuido en las cuentas de los países clasificados como paraíso fiscal y guardar este resultado en un `DataFrame` llamado `mfh_paraisos_fiscales`. 

Para esto declaramos la función `calcular_porcentaje`.

In [12]:
def calcular_porcentaje(lista):
    
    rta = lista / sum(lista)
    
    return rta 

Después la usamos como parámetro del método `apply`.

In [13]:
# Declaramos el Segmentador de Índices
idx = pd.IndexSlice

mfh_paraisos_fiscales = mfh.loc[idx[:,'Paraiso Fiscal',],]
mfh_paraisos_fiscales = mfh_paraisos_fiscales.apply(calcular_porcentaje)

mfh_paraisos_fiscales

,,Mes,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
,,Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Ireland,Paraiso Fiscal,Pertenece,0.207028,0.216058,0.212733,0.213056,0.213135,0.202215,0.187958,0.187285,0.187005,0.195208,0.199807,0.199399
Luxembourg,Paraiso Fiscal,Pertenece,0.172086,0.173207,0.170225,0.172612,0.172704,0.178188,0.170311,0.172715,0.175649,0.176304,0.180633,0.183749
Switzerland,Paraiso Fiscal,Pertenece,0.167366,0.163090,0.161222,0.159582,0.159819,0.161946,0.169273,0.161391,0.163879,0.164462,0.160977,0.162929
Hong Kong,Paraiso Fiscal,No pertenece,0.160941,0.161673,0.171768,0.171838,0.176846,0.174094,0.177163,0.177748,0.174134,0.172910,0.172219,0.170195
Belgium,Paraiso Fiscal,Pertenece,0.142979,0.138540,0.136270,0.141069,0.139439,0.141074,0.142630,0.144371,0.142130,0.143619,0.139596,0.143646
Singapore,Paraiso Fiscal,No pertenece,0.104235,0.103293,0.101994,0.097078,0.093419,0.098054,0.104844,0.109536,0.110606,0.102417,0.103869,0.096975
Netherlands,Paraiso Fiscal,Pertenece,0.045365,0.044139,0.045788,0.044766,0.044639,0.044430,0.047820,0.046954,0.046596,0.045080,0.042900,0.043108


#### Encadenar métodos 

En `pandas` podemos encadenar métodos, esto significa que se puede invocar un método tras otro sin redefinir el `DataFrame` repetidamente. El método `apply` puede encadenarse con otros métodos, lo que resulta práctico en términos de ahorro de líneas de código y claridad de lectura. 

##### Ejemplo 6

Filtra la base para solo trabajar con aquellos países que son parte de la OECD, de tal forma que pueda calcular el porcentaje de títulos bajo posesión de cada país.

Hacemos entonces el filtrado y directamente realizamos el `apply`.

In [14]:
mfh_oecd = mfh.loc[idx[:,:,'Pertenece'],].apply(calcular_porcentaje,axis = 0)
mfh_oecd

,,Mes,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
,,Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,0.346295,0.345299,0.348611,0.340431,0.344033,0.348428,0.349097,0.339497,0.336294,0.333055,0.331657,0.330346
United Kingdom,Tributación Regular,Pertenece,0.116381,0.113416,0.114505,0.120250,0.121684,0.118078,0.128847,0.127680,0.124965,0.113046,0.114448,0.116674
Ireland,Paraiso Fiscal,Pertenece,0.085692,0.090565,0.089188,0.089135,0.088492,0.082891,0.074505,0.075682,0.075401,0.081274,0.082786,0.080685
Luxembourg,Paraiso Fiscal,Pertenece,0.071229,0.072604,0.071367,0.072215,0.071705,0.073042,0.067510,0.069794,0.070822,0.073403,0.074841,0.074353
Switzerland,Paraiso Fiscal,Pertenece,0.069275,0.068363,0.067592,0.066764,0.066355,0.066384,0.067098,0.065218,0.066076,0.068473,0.066697,0.065928
Belgium,Paraiso Fiscal,Pertenece,0.059181,0.058072,0.057131,0.059019,0.057894,0.057828,0.056537,0.058340,0.057307,0.059795,0.057838,0.058125
France,Tributación Regular,Pertenece,0.036741,0.036086,0.035131,0.038914,0.035621,0.037883,0.042794,0.039500,0.037187,0.036817,0.035149,0.037713
Canada,Tributación Regular,Pertenece,0.034733,0.035383,0.036101,0.034407,0.035457,0.037498,0.036073,0.039687,0.043820,0.041315,0.044350,0.044414


#### Múltiples operaciones simultaneamente

El método `apply` permite ejecutar varias operaciones a la vez, lo cual nos ahorra líneas de código o encadenar más métodos de los necesarios. 

##### Ejemplo 7

Requerimos nuevamente filtrar la base para solo trabajar con aquellos países que son parte de la OECD, de tal forma que calculemos el porcentaje de títulos bajo posesión de cada país. Esta vez, redondearemos el resultado a tres cifras decimales. 

Una manera de solucionar este requerimiento es utilizar la división de objetos y el método `round` de `pandas`.

In [15]:
mfh_oecd = mfh.loc[idx[:,:,'Pertenece'],]
mfh_oecd_norm = (mfh_oecd / mfh_oecd.sum()).round(3)
mfh_oecd_norm

,,Mes,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
,,Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,0.346,0.345,0.349,0.340,0.344,0.348,0.349,0.339,0.336,0.333,0.332,0.330
United Kingdom,Tributación Regular,Pertenece,0.116,0.113,0.115,0.120,0.122,0.118,0.129,0.128,0.125,0.113,0.114,0.117
Ireland,Paraiso Fiscal,Pertenece,0.086,0.091,0.089,0.089,0.088,0.083,0.075,0.076,0.075,0.081,0.083,0.081
Luxembourg,Paraiso Fiscal,Pertenece,0.071,0.073,0.071,0.072,0.072,0.073,0.068,0.070,0.071,0.073,0.075,0.074
Switzerland,Paraiso Fiscal,Pertenece,0.069,0.068,0.068,0.067,0.066,0.066,0.067,0.065,0.066,0.068,0.067,0.066
Belgium,Paraiso Fiscal,Pertenece,0.059,0.058,0.057,0.059,0.058,0.058,0.057,0.058,0.057,0.060,0.058,0.058
France,Tributación Regular,Pertenece,0.037,0.036,0.035,0.039,0.036,0.038,0.043,0.040,0.037,0.037,0.035,0.038
Canada,Tributación Regular,Pertenece,0.035,0.035,0.036,0.034,0.035,0.037,0.036,0.040,0.044,0.041,0.044,0.044


En cambio podríamos haber usado una sola línea de código para cubrir este requerimiento.

In [16]:
mfh_oecd_norm = mfh.loc[idx[:,:,'Pertenece'],].apply(lambda x: round(x / x.sum(),3))
mfh_oecd_norm

,,Mes,Sep,Aug,Jul,Jun,May,Apr,Mar,Feb,Jan,Dec,Nov,Oct
,,Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,0.346,0.345,0.349,0.340,0.344,0.348,0.349,0.339,0.336,0.333,0.332,0.330
United Kingdom,Tributación Regular,Pertenece,0.116,0.113,0.115,0.120,0.122,0.118,0.129,0.128,0.125,0.113,0.114,0.117
Ireland,Paraiso Fiscal,Pertenece,0.086,0.091,0.089,0.089,0.088,0.083,0.075,0.076,0.075,0.081,0.083,0.081
Luxembourg,Paraiso Fiscal,Pertenece,0.071,0.073,0.071,0.072,0.072,0.073,0.068,0.070,0.071,0.073,0.075,0.074
Switzerland,Paraiso Fiscal,Pertenece,0.069,0.068,0.068,0.067,0.066,0.066,0.067,0.065,0.066,0.068,0.067,0.066
Belgium,Paraiso Fiscal,Pertenece,0.059,0.058,0.057,0.059,0.058,0.058,0.057,0.058,0.057,0.060,0.058,0.058
France,Tributación Regular,Pertenece,0.037,0.036,0.035,0.039,0.036,0.038,0.043,0.040,0.037,0.037,0.035,0.038
Canada,Tributación Regular,Pertenece,0.035,0.035,0.036,0.034,0.035,0.037,0.036,0.040,0.044,0.041,0.044,0.044


### 2.7. Método `rename`

El método `rename` sirve para renombrar columnas o índices. 

```python
    pandas.DataFrame.rename(mapper, index, columns, axis, inplace, level)
```

* **mapper:** diccionario o función para renombrar las columnas o índices. <br><br>

* **index:** diccionario o función para renombrar índices. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `mapper` y `axis`. <br><br>

* **columns:** diccionario o función para renombrar columnas. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `mapper` y `axis`. <br><br>

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`:  indica que vamos a renombrar columnas.<br><br>
    * `axis = 0`: indica que vamos a renombrar filas.<br><br>

* **inplace:** por defecto es `False`.<br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal.<br><br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones.<br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le debe aplicar el diccionario o la función. 

#### Ejemplo 8

En la siguiente celda encuentras un diccionario con los nombres de los países en inglés y en español.

In [17]:
df_paises = pd.read_csv("Archivos/DiccionarioPaises.csv", encoding = 'utf-8-sig', sep = ";", index_col = [1])
dicc_paises = df_paises.to_dict()['País en español']

A continuación encuentras un diccionario con las abreviaturas de los meses en inglés y en español.

In [18]:
dicc_meses = {'Jan':'ENE', 'Feb': 'FEB', 'Mar':'MAR', 'Apr':'ABR', 'May':'MAY' ,'Jun': 'JUN', 'Jul':'JUL',
                    'Aug': 'AGO', 'Sep': 'SEP', 'Oct':'OCT','Nov':'NOV', 'Dec':'DIC'}

Requerimos traducir al español los meses y países del `DataFrame` utilizando los diccionarios provistos 

In [19]:
mfh = mfh.rename(mapper = dicc_paises)
mfh = mfh.rename(columns = dicc_meses) 
mfh

,,Mes,SEP,AGO,JUL,JUN,MAY,ABR,MAR,FEB,ENE,DIC,NOV,OCT
,,Año,2020,2020,2020,2020,2020,2020,2020,2020,2020,2019,2019,2019
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japón,Tributación Regular,Pertenece,1276.2,1278.4,1293.0,1261.5,1260.4,1266.5,1272.6,1268.6,1211.8,1155.2,1160.6,1168.5
China,Tributación Regular,No pertenece,1061.7,1068.0,1073.4,1074.4,1083.7,1072.8,1081.6,1092.3,1078.6,1069.9,1089.1,1101.5
Reino Unido,Tributación Regular,Pertenece,428.9,419.9,424.7,445.6,445.8,429.2,469.7,477.1,450.3,392.1,400.5,412.7
Irlanda,Paraiso Fiscal,Pertenece,315.8,335.3,330.8,330.3,324.2,301.3,271.6,282.8,271.7,281.9,289.7,285.4
Brasil,Tributación Regular,No pertenece,265.1,265.0,265.7,264.1,264.4,259.5,264.4,285.9,283.3,281.8,293.3,298.5
Luxemburgo,Paraiso Fiscal,Pertenece,262.5,268.8,264.7,267.6,262.7,265.5,246.1,260.8,255.2,254.6,261.9,263.0
Suiza,Paraiso Fiscal,Pertenece,255.3,253.1,250.7,247.4,243.1,241.3,244.6,243.7,238.1,237.5,233.4,233.2
Hong kong,Paraiso Fiscal,No pertenece,245.5,250.9,267.1,266.4,269.0,259.4,256.0,268.4,253.0,249.7,249.7,243.6


### 2.8. Método `replace`

El método `replace` usa la siguiente sintaxis para reemplazar un valor por otro.

```python
pandas.DataFrame.replace( to_replace, value, inplace) 
```

* **to_replace:** valor específico, estructura de datos. <br><br>

* **value:** valor que sustituirá el valor a reemplazar.<br><br>

* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal.<br><br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

### 2.9. Método `stack`

El método `stack` transfiere uno o varios niveles de las columnas al índice. Al emplearlo, el número de niveles del índice se va a ver aumentando y el número de niveles de las columnas se va a ver disminuido. 

```python
pandas.DataFrame.stack(level, fill_value) 
```
 
* **level:** nombre o índice del nivel a transferir. Por defecto es -1 (el último nivel). <br><br>

* **fill_value:** valor para reemplazar los datos faltantes que se generen.

#### Ejemplo 9

Pasa el nivel correspondiente a los meses al índice de `mfh_2021`.

Procedemos entonces usando el método `stack`.

In [20]:
mfh_2021 = mfh_2021.stack("Mes") 
mfh_2021

Año                                                           2019
Pais                 Clasificación_Fiscal OECD         Mes        
Japan                Tributación Regular  Pertenece    Dec  1155.2
                                                       Nov  1160.6
                                                       Oct  1168.5
China, Mainland      Tributación Regular  No pertenece Dec  1069.9
                                                       Nov  1089.1
...                                                            ...
United Arab Emirates Tributación Regular  No pertenece Nov    39.9
                                                       Oct    38.4
Vietnam              Tributación Regular  No pertenece Dec    30.5
                                                       Nov    29.9
                                                       Oct    29.2

[99 rows x 1 columns]

### 2.10. Método `unstack`

El método `unstack` transfiere uno o varios niveles del índice a las columna. Al emplearlo, el número de niveles de las columnas se va a ver aumentando y el número de niveles del índice se va a ver disminuido. 

```python
pandas.DataFrame.unstack(level, fill_value) 
```

* **level:** nombre o índice del nivel a transferir. Por defecto es -1 (el último nivel). <br><br>

* **fill_value:** valor para reemplazar los datos faltantes que se generen.

#### Ejemplo 10

Se te solicita revertir el cambio realizado en el ejemplo anterior.

Utilizamos el método `unstack` para revertir el cambio realizado.

In [21]:
mfh_2021 = mfh_2021.unstack("Mes")
mfh_2021

Año                                                       2019                
Mes                                                        Dec     Nov     Oct
Pais                 Clasificación_Fiscal OECD                                
Australia            Tributación Regular  Pertenece       41.8    44.6    45.4
Belgium              Paraiso Fiscal       Pertenece      207.4   202.4   205.6
Bermuda              Tributación Regular  No pertenece    71.3    68.2    65.4
Brazil               Tributación Regular  No pertenece   281.8   293.3   298.5
Canada               Tributación Regular  Pertenece      143.3   155.2   157.1
Cayman Islands       Tributación Regular  No pertenece   238.2   234.2   236.8
China, Mainland      Tributación Regular  No pertenece  1069.9  1089.1  1101.5
France               Tributación Regular  Pertenece      127.7   123.0   133.4
Germany              Tributación Regular  Pertenece       78.3    79.9    84.0
Hong Kong            Paraiso Fiscal       No pertenece   249.7   249.7   243.6
India                Tributación Regular  No pertenece   162.0   159.2   161.4
Ireland              Paraiso Fiscal       Pertenece      281.9   289.7   285.4
Israel               Tributación Regular  Pertenece       43.1    45.6    45.4
Italy                Tributación Regular  Pertenece       45.3    45.7    47.1
Japan                Tributación Regular  Pertenece     1155.2  1160.6  1168.5
Korea                Tributación Regular  Pertenece      121.9   117.2   117.1
Kuwait               Tributación Regular  No pertenece    43.3    43.0    43.8
Luxembourg           Paraiso Fiscal       Pertenece      254.6   261.9   263.0
Mexico               Tributación Regular  Pertenece       45.3    49.1    49.1
Netherlands          Paraiso Fiscal       Pertenece       65.1    62.2    61.7
Norway               Tributación Regular  Pertenece       90.1    96.0   100.3
Philippines          Tributación Regular  No pertenece    34.8    37.4    34.6
Poland               Tributación Regular  Pertenece       38.8    38.5    35.7
Saudi Arabia         Tributación Regular  No pertenece   179.8   179.7   178.9
Singapore            Paraiso Fiscal       No pertenece   147.9   150.6   138.8
Spain                Tributación Regular  Pertenece       50.4    44.4    45.6
Sweden               Tributación Regular  Pertenece       48.7    49.5    46.9
Switzerland          Paraiso Fiscal       Pertenece      237.5   233.4   233.2
Taiwan               Tributación Regular  No pertenece   193.1   188.6   191.2
Thailand             Tributación Regular  No pertenece    90.5    91.2    95.2
United Arab Emirates Tributación Regular  No pertenece    40.7    39.9    38.4
United Kingdom       Tributación Regular  Pertenece      392.1   400.5   412.7
Vietnam              Tributación Regular  No pertenece    30.5    29.9    29.2

## Referencias

Pandas (2021). Documentación sobre I/O tools (text, CSV, HDF5...). Recuperado el 24 de Abril de 2020 de: https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html 

Pandas (2020). Documentación sobre los objetos índices. Recuperado el 15 de mayo de 2020 de: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html

Pandas (2020). Documentación sobre los métodos DataFrame. Recuperado el 15 de mayo de 2020 de: https://pandas.pydata.org/docs/reference/frame.html

Departamento del Tesoro Americano (2020). Principales tenedores foráneos de títulos del tesoro [Base de Datos]. Recuperado el  15 de septiembre de 2020 de : 
https://ticdata.treasury.gov/Publish/mfh.txt.

## Créditos

**Autores:** Jorge Esteban Camargo Forero, Alejandro Mantilla Redondo, Diego Alejandro Cely Gómez

**Fecha Última actualización:** 12/07/2022